In [18]:
import heapq

class NetworkNode:
    def __init__(self, ID, latitude, longitude):
        self.ID = ID
        self.latitude = latitude
        self.longitude = longitude

    def __repr__(self):
        return f"{self.__class__.__name__}({self.ID})"

    def __hash__(self):
        return hash(self.ID)

    def __eq__(self, other):
        return self.ID == other.ID

# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
class FogCloudNode(NetworkNode):
    def __init__(self, ID, longitude, latitude, processing_capacity, memory_capacity, cost, model):
        super().__init__(ID, latitude, longitude)
        self.model = model
        self.processing_capacity = processing_capacity
        self.memory_capacity = memory_capacity
        self.cost = cost

class FogNode(FogCloudNode):
    pass

class CloudNode(FogCloudNode):
    pass

# Sensores -> index | longitude | latitude | services
class Sensor(NetworkNode):
    def __init__(self, ID, longitude, latitude, services):
        super().__init__(ID, latitude, longitude)
        self.services = services

class Grafo:
    def __init__(self):
        self.adj = {}
        self.n_vertices = 0
        self.n_arestas = 0

    def add_vertice(self, noh):
        if noh not in self.adj:
            self.adj[noh] = []
            self.n_vertices += 1

    def add_aresta(self, u, v, largura_banda, custo, dist_H):
        self.add_vertice(u)
        self.add_vertice(v)
        # Tempo de delay = latência (valor obtido da tecnologia) + propagação (harvesine/2*10^8)
        tempo = 0.001 + dist_H/(2*10**8)
        self.adj[u].append((v, largura_banda, custo, tempo))
        self.n_arestas += 1

    def copiar(self):
        novo = Grafo()
        novo.n_arestas =  self.n_arestas
        for vertice, arestas in self.adj.items():
            novo.add_vertice(vertice)
            for v, largura_banda, custo, tempo in arestas:
                novo.adj[vertice].append((v, largura_banda, custo, tempo))
        return novo

def dijkstra(grafo, origem):
    tempo = {n: float('inf') for n in grafo.adj}
    tempo[origem] = 0
    prev = {n: None for n in grafo.adj}
    fila = [(0, origem)]

    while fila:
        tempo_atual, u = heapq.heappop(fila)
        if tempo_atual > tempo[u]:
            continue

        for v, _, _, t in grafo.adj[u]:
            novo_tempo = tempo[u] + t 
            if novo_tempo < tempo[v]:
                tempo[v] = novo_tempo
                prev[v] = u
                heapq.heappush(fila, (novo_tempo, v))

    return tempo, prev

def reconstruir_caminho(prev, destino):
    caminho = []
    while destino is not None:
        caminho.append(destino)
        destino = prev[destino]
    return caminho[::-1]

def processar_caminho(grafo, caminho, requisicao):
    for i in range(len(caminho)-1):
        vertice_atual = caminho[i]
        vizinhos = grafo.adj[vertice_atual]
        # Selecionar o vizinho correto
        selecionado = None
        for vizinho in vizinhos:
            if(vizinho[0] == caminho[i+1]):
                selecionado = vizinho
                break
        if selecionado is None:
            return False
        if(requisicao[0] <= selecionado[0].processing_capacity):
            if(requisicao[1] <= selecionado[0].memory_capacity):
                selecionado[0].processing_capacity -= requisicao[0]
                selecionado[0].memory_capacity -= requisicao[1]
                return True
    return False    
        

# Próximos passos - leitura da instância na main
# Leitura e armazenamentos dos dados até "#end_services"
    # instance_info -> cabeçalho (apenas algumas informações são importantes
    # sensors -> ler e armazenar numa lista de objetos Sensor()
    # reach_fog_nodes -> ler e armazenar num grafo as arestas Sensor-Fog
    # fog -> ler e armazenar numa lista de objetos FogNode() e arestas Fog-Fog
    # cloud -> ler e armazenar numa lista de objetos CloudNode() e arestas Fog-Cloud
    # service -> ler e armazenar num dicionário de services, com a chave sendo o nome do serviço
##
# Começa a leitura das requests
    # Para cada time_instant
        # Armazena o valor do time instant numa variável i
        # Verifica na ED temporal se naquele i precisa de alteração no grafo (retorno de disponibilidade de capacidade)
        # Se sim, realiza todas as alterações necessárias
        # Roda o Dijkstra
        # Começa a percorrer as requisições do tipo "sensor ID | service index | type of service | request_lifetime"
        # Chama a função processar_caminho() para essa requisição
        # Se True
            # ED Temporal: adiciona para o instante i+request_lifetime+1 o valor (FogCloudNode, services[type_of_service][0], services[type_of_service][1])
            # Numero de requisições aceitas: incrementado
            # Número de requisições do tipo "type of service" aceitas: incrementado
            # Dentro de processar_caminho()
                # Sinaliza o nó fog em que foi aceita a requisição
                # Conta a quantidade de arcos utilizados do sensor inicial até o nó fog em que foi processado
                # Soma a quantidade de largura de banda 
                # Soma o custo dos arcos com o custo do processamento
                # Retorna todos esse valores
# Após o "#end_requests", retorna todos os valores de dados de análise e gera-se o mapa de calor 

# Criando os nós
# Sensores -> index | longitude | latitude | services
# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
sensor = Sensor("S1", 0.0, 0.0, services=[])
fog1 = FogNode("F1", 1.0, 1.0, 80, 16, 5, "model-A")
fog2 = FogNode("F2", 1.5, 1.5, 70, 8, 6, "model-B")
cloud = CloudNode("C1", 2.0, 2.0, 500, 128, 15, "cloud-X")

# Criando o grafo e adicionando conexões
g = Grafo()

# Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
g.add_aresta(sensor, fog1, 3, 4, 12)     
g.add_aresta(sensor, fog2, 5, 6, 30)     
g.add_aresta(fog1, fog2, 4, 5, 40)       
g.add_aresta(fog2, fog1, 6, 5, 12)      
g.add_aresta(fog1, cloud, 4, 8, 90)      
g.add_aresta(fog2, cloud, 4, 4, 120)      

# ID: [processing_demand, memory_demand, number_of_bits, lifetime]
services = {
    'waste': [0.2125, 0.375, 296, 50],
    'camera': [0.35, 0.475, 12000, 10],
    'air': [0.25, 0.3125, 744, 100]
}
tempo, prev = dijkstra(g, sensor)
print(tempo)
print(prev)
caminho = reconstruir_caminho(prev, cloud)
print(caminho)
print(g.adj)
print("#"*30)
print(processar_caminho(g.copiar(), caminho, services["waste"]))


{Sensor(S1): 0, FogNode(F1): 0.00100006, FogNode(F2): 0.00100015, CloudNode(C1): 0.00200051}
{Sensor(S1): None, FogNode(F1): Sensor(S1), FogNode(F2): Sensor(S1), CloudNode(C1): FogNode(F1)}
[Sensor(S1), FogNode(F1), CloudNode(C1)]
{Sensor(S1): [(FogNode(F1), 3, 4, 0.00100006), (FogNode(F2), 5, 6, 0.00100015)], FogNode(F1): [(FogNode(F2), 4, 5, 0.0010002), (CloudNode(C1), 4, 8, 0.00100045)], FogNode(F2): [(FogNode(F1), 6, 5, 0.00100006), (CloudNode(C1), 4, 4, 0.0010006)], CloudNode(C1): []}
##############################
True
